# Clasificador de Vinos con KNN

Entrena un modelo de K-Vecinos más Cercanos (KNN) para predecir la calidad de un vino tinto a partir de sus características químicas. ¿Podría una IA ayudarte a elegir un vino digno de sommelier?

Utilizaremos el siguiente dataset de vinos tintos extraido de Wine Quality Data Set - UCI: 

https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/refs/heads/main/winequality-red.csv

### Descripción de las columnas

Cada fila representa un vino. Las columnas describen su composición química:
* fixed acidity, volatile acidity, citric acid
* residual sugar, chlorides
* free sulfur dioxide, total sulfur dioxide
* density, pH, sulphates, alcohol

La columna objetivo es label:
* 0 = Baja calidad
* 1 = Calidad media
* 2 = Alta calidad
